In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install vaderSentiment

import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load your uploaded CSV
df = pd.read_csv("youtube_helene_comments.csv")

# Fill NaNs in the relevant column
df['text'].fillna('', inplace=True)

# Use only the 'text' column for analysis
df["combined_text"] = df["text"]

keywords = [
    "insurance", "claim", "claims", "damage", "economic loss", "flood",
    "power outage", "state farm", "allstate", "duke energy", "adjuster",
    "home damage", "roof", "loss", "coverage", "policy", "fema", "repair"
]

def is_relevant(text):
    text = text.lower()
    return any(keyword in text for keyword in keywords)

df["relevant"] = df["combined_text"].apply(is_relevant)
df_relevant = df[df["relevant"]].copy() # Use .copy() to avoid SettingWithCopyWarning


analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    score = analyzer.polarity_scores(text)["compound"]
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"

df_relevant["sentiment"] = df_relevant["combined_text"].apply(get_sentiment)

df_relevant.to_csv("youtube_sentiment_results.csv", index=False)

# Download the result
from google.colab import files
files.download("youtube_sentiment_results.csv")